<a href="https://colab.research.google.com/github/wgova/time_series_trade/blob/master/kmeans_basecode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [2]:
!pip install tslearn
import pandas as pd
import numpy as np
import glob
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

import matplotlib.pyplot as plt
%matplotlib inline

     |████████████████████████████████| 747kB 3.2MB/s 


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
import os
os.chdir('/content/drive/My Drive/phd_data/sitc/')
os.getcwd()

'/content/drive/My Drive/phd_data/sitc'

# Functions

In [0]:
def read_filedata(filenames):
    dfs = []
    for filename in filenames:
        original_df = pd.read_csv(filename, index_col=None, header=0)
        dfs.append(original_df)
    return dfs

def time_series_data(dataframes, target_col=None):
    tsdata = []
    for i, df in enumerate(dataframes):
        tsdata.append(df[target_col].values.tolist()[:])
        len_max = 0
        for ts in tsdata:
            if len(ts) > len_max:
                len_max = len(ts)
        for i, ts in enumerate(tsdata):
            len_add = len_max - len(ts)
            tsdata[i] = ts + [ts[-1]] * len_add
    
    tsdata = np.array(tsdata)
    return tsdata

def transform_vector(time_series_array):
    stack_list = []
    for j in range(len(time_series_array)):
        data = np.array(time_series_array[j])
        data = data.reshape((1, len(data))).T
        stack_list.append(data)
    stack_data = np.stack(stack_list, axis=0)
    return stack_data

# Files

In [0]:
files = os.listdir()

In [6]:
countries = pd.read_csv('list_countries.csv',usecols=['id','name'],
                        na_values='',na_filter=True)
countries.rename(columns=lambda x: x.replace('name','country'), inplace=True)
countries.rename(columns=lambda x: x.replace('id','origin_id'), inplace=True)
countries.columns

Index(['origin_id', 'country'], dtype='object')

In [0]:
list_df = []
def create_df(product_code):
  exclude_list = ['export_rca','export_val_growth_pct','export_val_growth_pct_5', 'export_val_growth_val','export_val_growth_val_5', 
                'import_rca', 'import_val','import_val_growth_pct', 'import_val_growth_pct_5','import_val_growth_val', 'import_val_growth_val_5','sitc_id','sitc_id_len']
  #trade_data = [i for i in os.listdir(os.getcwd()) if i.endswith(f'{product_code}.csv') and 'sitc' in i]
  #for file in trade_data:
  #  trade_history = []
  #  file_name = pd.read_csv(file).drop(exclude_list, axis=1)
  #  trade_history.append(file_name)
  #products_dataframe = pd.concat(trade_history)
  product_df = pd.read_csv(product_code,index_col=None, header=0).drop(exclude_list, axis=1)
  trade_dframe = product_df.merge(countries, on=['origin_id'],how='left')
  trade_dframe = trade_dframe.drop(['origin_id'],axis=1)
  trade_dframe = trade_dframe[trade_dframe['export_val'].notna()]
  return trade_dframe

In [66]:
xtrade_years = create_df('sitc-2924.csv')
xtrade_years.set_index(["year"])
groups = xtrade_years.groupby([xtrade_years.year.name,xtrade_years.country.name]).mean().unstack()
groups.columns = groups.columns.droplevel()
# xtrade_years['year'] = pd.to_datetime(xtrade_years['year'], format='%Y')
# xtrade_years.apply(pd.to_numeric, errors='coerce').fillna(0, downcast='infer',inplace=True)
# groups.replace('NaN',0, inplace=True)
# groups.dropna(thresh = 0, how='any',axis = 1, inplace = True)
groups.isnull().sum()
groups.head()

country,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,Antigua and Barbuda,Areas,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belgium-Luxembourg,Belize,Benin,Bermuda,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Bouvet Island,Brazil,British Indian Ocean Territory,British Virgin Islands,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cambodia,...,Sudan,Suriname,Swaziland,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Tokelau,Tonga,Trinidad and Tobago,Tunisia,Turkey,Turkmenistan,Turks and Caicos Islands,Tuvalu,USSR,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,United States Minor Outlying Islands,United States Unspecified,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Wallis and Futuna,Western Sahara,Yemen,Yemen Arab Republic,Yugoslavia,Zambia,Zimbabwe
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1962.0,15000.0,151000.0,464000.0,NaN,NaN,91000.0,NaN,NaN,NaN,1306000.0,103000.0,NaN,NaN,1102000.0,174000.0,NaN,27000.0,NaN,NaN,NaN,NaN,329000.0,NaN,NaN,12000.0,12000.0,NaN,130000.0,NaN,NaN,NaN,1374000.0,NaN,NaN,NaN,1704000.0,5000.0,189000.0,86000.0,104000.0,...,600000.0,1000.0,NaN,10000.0,175000.0,82000.0,463000.0,NaN,113000.0,475000.0,NaN,NaN,NaN,NaN,133000.0,42000.0,572000.0,NaN,NaN,NaN,920000.0,NaN,NaN,NaN,518000.0,4810000.0,NaN,1000.0,NaN,NaN,NaN,436000.0,559000.0,NaN,NaN,NaN,NaN,1936000.0,14000.0,NaN
1963.0,385000.0,116000.0,508000.0,NaN,NaN,137000.0,NaN,NaN,NaN,1203000.0,369000.0,NaN,NaN,955000.0,106000.0,NaN,26000.0,NaN,NaN,NaN,NaN,395000.0,NaN,NaN,2000.0,41000.0,NaN,136000.0,NaN,NaN,NaN,1901000.0,NaN,NaN,NaN,1725000.0,3000.0,56000.0,10000.0,158000.0,...,1365000.0,5000.0,NaN,57000.0,173000.0,312000.0,833000.0,NaN,120000.0,347000.0,NaN,1000.0,NaN,NaN,87000.0,54000.0,1652000.0,NaN,NaN,NaN,1263000.0,NaN,NaN,NaN,548000.0,5652000.0,NaN,NaN,4000.0,NaN,NaN,160000.0,611000.0,NaN,NaN,NaN,3000.0,2068000.0,7000.0,NaN
1964.0,627000.0,130000.0,363000.0,NaN,NaN,199000.0,NaN,NaN,NaN,2262000.0,294000.0,NaN,NaN,888000.0,178000.0,NaN,11000.0,NaN,NaN,NaN,NaN,310000.0,NaN,NaN,26000.0,26000.0,NaN,199000.0,NaN,NaN,NaN,1830000.0,NaN,NaN,NaN,1346000.0,15000.0,46000.0,103000.0,98000.0,...,1943000.0,NaN,NaN,52000.0,360000.0,162000.0,1022000.0,NaN,997000.0,478000.0,NaN,1000.0,NaN,NaN,123000.0,53000.0,642000.0,NaN,NaN,NaN,979000.0,60000.0,NaN,NaN,692000.0,5871000.0,NaN,NaN,NaN,NaN,NaN,92000.0,169000.0,NaN,NaN,NaN,4000.0,2395000.0,9000.0,NaN
1965.0,272000.0,112000.0,399000.0,NaN,NaN,221000.0,NaN,NaN,NaN,3609000.0,313000.0,NaN,NaN,1008000.0,80000.0,NaN,15000.0,NaN,NaN,NaN,NaN,407000.0,NaN,NaN,3000.0,9000.0,NaN,533000.0,NaN,NaN,NaN,1308000.0,NaN,NaN,NaN,1143000.0,2000.0,41000.0,304000.0,91000.0,...,1603000.0,NaN,NaN,70000.0,296000.0,116000.0,1047000.0,NaN,1980000.0,865000.0,NaN,44000.0,NaN,NaN,81000.0,67000.0,861000.0,NaN,NaN,NaN,1398000.0,33000.0,NaN,NaN,351000.0,5627000.0,NaN,1000.0,NaN,NaN,NaN,29000.0,196000.0,NaN,NaN,NaN,1000.0,2650000.0,9000.0,NaN
1966.0,159000.0,144000.0,358000.0,NaN,NaN,319000.0,NaN,NaN,NaN,5801000.0,314000.0,NaN,NaN,1440000.0,95000.0,NaN,17000.0,NaN,NaN,3000.0,NaN,609000.0,NaN,NaN,4000.0,12000.0,NaN,2341000.0,NaN,NaN,NaN,875000.0,NaN,NaN,NaN,1552000.0,NaN,117000.0,306000.0,102000.0,...,1423000.0,2000.0,NaN,34000.0,286000.0,86000.0,1888000.0,NaN,2067000.0,661000.0,NaN,53000.0,NaN,NaN,94000.0,58000.0,621000.0,NaN,NaN,NaN,1241000.0,7000.0,NaN,NaN,962000.0,7280000.0,NaN,NaN,NaN,NaN,NaN,31000.0,108000.0,NaN,NaN,NaN,NaN,2867000.0,21000.0,NaN


In [0]:
def group_ts_by_country(df,threshold):
  grp = df.set_index('year')
  grp = grp.apply(pd.to_numeric, errors='coerce').fillna(0, downcast='infer')
  grp = df.groupby([df.year.name,df.country.name]).mean().unstack()
  grp.columns = grp.columns.droplevel()
  thresh = int(len(grp) * threshold)
  grp.dropna(thresh = thresh, axis = 1, inplace = True)
  return grp

In [11]:
df_grps =group_ts_by_country(xtrade_years,0)

AttributeError: ignored

In [0]:
# xtrade_years.set_index(["country","year"]).squeeze()

In [72]:
# filenames = sorted(glob.glob('sitc-*.csv'))
# trade_dframe['export_val'].replace('',0, inplace=True)
billion = 1000000000
xtrade_years['export_val'] = xtrade_years['export_val']/billion
# df = trade_dframe[trade_dframe['export_val'].notna()]
# df=df.set_index(['sitc_id',"country","year"])
# df.drop(['origin_id'])
# dfa = df.squeeze()
# df.isnull().sum()
# df.head()
xtrade_years['export_val'].astype(int)

0       0
1       0
2       0
3       0
4       0
       ..
8615    0
8618    0
8619    0
8620    0
8621    0
Name: export_val, Length: 8511, dtype: int64

Reference: https://github.com/masatakashiwagi/Analysis_tslearn